<a href="https://colab.research.google.com/github/tnc-br/ddf-isoscapes/blob/kriging_ensemble_data/dnn/kriging_ensemble_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
MEAN_PREDICTED_NAME = 'ordinary_kriging_linear_d18O_predicted_mean' #@param
VAR_PREDICTED_NAME = 'ordinary_kriging_linear_d18O_predicted_variance' #@param

INPUT_PATH = 'canonical/uc_davis_2023_08_26_isoscapes_test_fixed_grouped.csv' #@param

MEANS_ISOSCAPE_PATH = 'canonical/uc_davis_d18O_cel_ordinary_linear_fixed_grouped_means.tiff' #@param
VARS_ISOSCAPE_PATH = 'canonical/uc_davis_d18O_cel_ordinary_linear_fixed_grouped_vars.tiff' #@param

OUTPUT_PATH = 'kriging_ensemble_experiment/uc_davis_2023_08_26_isoscapes_test_fixed_grouped_ordinary_kriging_linear.csv' #@param


In [2]:
#@title Imports and modules.
%pip install opencv-python
%pip install matplotlib
%pip install pandas

from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import matplotlib.animation as animation
from matplotlib import rc
from typing import List
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
from tqdm import tqdm
from io import StringIO
import xgboost as xgb
import os
import math
import glob

rc('animation', html='jshtml')

import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
# ddfimport.ddf_source_control_pane()
ddfimport.ddf_import_common()

Cloning into 'ddf_common_stub'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 5.50 KiB | 5.50 MiB/s, done.
Resolving deltas: 100% (4/4), done.
executing checkout_branch ...
b''
main branch checked out as readonly. You may now use ddf_common imports


In [3]:
import dataset

In [4]:
import raster

In [6]:
raster.mount_gdrive()

Mounted at /content/gdrive


In [28]:
df = pd.read_csv(raster.get_sample_db_path(INPUT_PATH), index_col=0)

In [13]:
means_isoscape = raster.load_raster(raster.get_raster_path(MEANS_ISOSCAPE_PATH), use_only_band_index=0)
vars_isoscape = raster.load_raster(raster.get_raster_path(VARS_ISOSCAPE_PATH), use_only_band_index=0)

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)


In [30]:
df[MEAN_PREDICTED_NAME] = df.apply(lambda row:raster.get_data_at_coords(means_isoscape, row['long'],row['lat'],-1), axis=1)
df[VAR_PREDICTED_NAME] = df.apply(lambda row:raster.get_data_at_coords(vars_isoscape, row['long'],row['lat'],-1), axis=1)

In [32]:
df.to_csv(raster.get_sample_db_path(OUTPUT_PATH))